In [1]:
import os
import PyPDF2
from unidecode import unidecode

In [2]:
folder_path = "C:/Users/User/Documents/GitHub/CNHGitHub/Data Perú/Conflictos sociales/pdf_reports_conflictos_sociales/"
#folder_path = "D:/1. Documentos/0. Bases de datos/0. Dofiles y Scripts/CNH_rep/Data Perú/Conflictos sociales/pdf_reports_conflictos_sociales/"

def extraeTextoPdf(pdfPath, textIni, textFin):
    with open(pdfPath, 'rb') as pdfFile:
        # Create a PDF reader object
        pdfReader = PyPDF2.PdfReader(pdfFile)
    
        # Iterate through each page in the PDF
        contPagesIni = 0
        for pageNum in range(len(pdfReader.pages)):
            # Extract text from the current page
            page = pdfReader.pages[pageNum]
            testText = page.extract_text()
            if textIni in testText:
                break
            else:
                contPagesIni += 1

        contPagesFin = 0
        for pageNum in range(len(pdfReader.pages)):
            # Extract text from the current page
            page = pdfReader.pages[pageNum]
            testText = page.extract_text()
            if textFin in testText:
                break
            else:
                contPagesFin += 1
        
        # Initialize a variable to store the extracted text
        extracted_text = ''
        for pageNum in range(contPagesIni,contPagesFin):
            # Extract text from the current page
            page2 = pdfReader.pages[pageNum]
            extracted_text += page2.extract_text()
    
    return extracted_text
       

In [72]:
# Open the PDF file in read-binary mode
pdfPath = folder_path + '234_202_3.pdf'

# En un solo departamento
conflictosActivos1 = extraeTextoPdf(pdfPath = pdfPath, textIni = "5.1 CONFLICTOS ACTIVOS", textFin= "5.2 CONFLICTOS")

# En más de un departamento
conflictosActivos2  = extraeTextoPdf(pdfPath = pdfPath, textIni = "5.2 CONFLICTOS", textFin= "5.3 CONFLICTOS")

# Reactivados
conflictosReactivados = extraeTextoPdf(pdfPath = pdfPath, textIni = "5.3 CONFLICTOS", textFin= "VI. DETALLE DE LOS CONFLI")

# Latentes
conflictosLatentes = extraeTextoPdf(pdfPath = pdfPath, textIni = "VI. DETALLE DE LOS CONFLI", textFin= "6.1 CONFLICTOS")


In [73]:
encabezados = ["Reporte Mensual de Conflictos Sociales N.° 234 – Agosto 2023", "ADJUNTÍA PARA LA  PREVENCIÓN DE CONFLICTOS SOCIALES Y LA GOBERNABILIDAD – DEFENSORÍA DEL PUEBL O"]
for word in encabezados:
    conflictosActivos1 = conflictosActivos1.replace(word, "")

conflictosActivos1 = unidecode(conflictosActivos1)

In [74]:
departamentosPeru = [
    "Amazonas",
    "Ancash",
    "Apurimac",
    "Arequipa",
    "Ayacucho",
    "Cajamarca",
    "Callao",
    "Cusco",
    "Huancavelica",
    "Huanuco",
    "Ica",
    "Junin",
    "La Libertad",
    "Lambayeque",
    "Lima",
    "Loreto",
    "Madre de Dios",
    "Moquegua",
    "Pasco",
    "Piura",
    "Puno",
    "San Martin",
    "Tacna",
    "Tumbes",
    "Ucayali"
]

In [75]:
print(conflictosActivos1)

    
 
 26 V. DETALLE DE LOS CONFLI CTOS SOCIALES ACTIVO S 
Observacion: Los conflictos sociales en mas de un departamento se consideran como uno solo en el numero 
total de conflictos. Ver el detalle de los conflictos en mas de un departamento en la pa gina 104. 
5.1 CONFLICTOS ACTIVOS DE SARROLLADOS EN UN SO LO DEPARTAMENTO  
Amazonas  
Descripcion  Hechos del mes  
Tipo:  Socioambiental.  
Ingreso como caso nuevo: Julio de 2017.  
Caso: Comunidades Nativas Wampis exigen a mineros ilegales 
el cese de sus actividades y el retiro inmediato de su territorio. 
En atencion a dicha situacion, tambien le exigen al Estado se 
implementen estrategias de lucha contra dicha actividad.  
Ubicacion:  Distrito de Rio Santiago, provincia de Condorcanqui, 
region Amazonas.  
Actores primarios: Gobierno Territorial Autonomo de la Nacion 
Wampis - GTANW (integrado por las comunidades Wampis de 
las cuencas Santiago y Morona), mineros ilegales, poblacion del 
Centro Poblado La Poza.  
Actores secundar

In [88]:
results = {}

# Identificar si los departamentos tienen conflictos
dptos = []
for dpto in departamentosPeru:
    try:
        encuentra = conflictosActivos1.index(dpto + "  ")
        dptos.append(dpto)
    except:
        continue

# Agrupación por dptos
nDpto = 0
for dpto in dptos:
    if dpto == dptos[-1]:
        inicio = conflictosActivos1.index(dptos[nDpto] + "  ")
        fin = len(conflictosActivos1)
        results[dpto] = {conflictosActivos1[inicio + len(dptos[nDpto] + "  ")+1 :fin]}
    else:
        inicio = conflictosActivos1.index(dptos[nDpto] + "  ")
        fin = conflictosActivos1.index(dptos[nDpto + 1] + "  ")
        results[dpto] = {conflictosActivos1[inicio + len(dptos[nDpto] + "  ")+1 :fin]}
        nDpto = nDpto + 1

    stopWords=["Tipo:","\nIngreso como caso nuevo:", "\nCaso:","\nUbicacion:","Actores primarios:", "Actores secundarios:","Actores terciarios:","NO HAY DIALOGO", "HAY DIALOGO"]
    textoDpto = list(results[dpto])[0]

    textoDpto = textoDpto.replace("Descripcion  Hechos del mes", "")
    textoDpto = textoDpto.replace("Hechos del mes  Hechos del mes", "")
    textoDpto = textoDpto.replace("\nCASO NUEVO", "")
    textoDpto = textoDpto.replace("\nCaso :", "\nCaso:")
    textoDpto = textoDpto.replace("\nTipo :", "\nTipo:")
    textoDpto = textoDpto.replace("\nActores primarios :", "\nActores primarios:")
    textoDpto = textoDpto.replace("\nActores secundarios :", "\nActores secundarios:")
    textoDpto = textoDpto.replace("\nActores terciarios :", "\nActores terciarios:")
    textoDpto = textoDpto.replace("\nUbicacion :", "\nUbicacion:")

    for word in stopWords:
        if word == "NO HAY DIALOGO" :
            textoDpto = textoDpto.replace(word, "|" + "NOHAYDIALOGO" + "|")
        elif word == "HAY DIALOGO":
            textoDpto = textoDpto.replace(word, "|" + "HAYDIALOGO" + "|")
        else:
            textoDpto = textoDpto.replace(word, "|" + word)


    results[dpto] = textoDpto
    

In [79]:
print(texto)

  
Tipo:  Socioambiental.  
Ingreso como caso nuevo: Julio de 2017.  
Caso: Comunidades Nativas Wampis exigen a mineros ilegales 
el cese de sus actividades y el retiro inmediato de su territorio. 
En atencion a dicha situacion, tambien le exigen al Estado se 
implementen estrategias de lucha contra dicha actividad.  
Ubicacion:  Distrito de Rio Santiago, provincia de Condorcanqui, 
region Amazonas.  
Actores primarios: Gobierno Territorial Autonomo de la Nacion 
Wampis - GTANW (integrado por las comunidades Wampis de 
las cuencas Santiago y Morona), mineros ilegales, poblacion del 
Centro Poblado La Poza.  
Actores secundarios: Gobierno Regional de Amazonas (GORE 
Amazonas), Coordinadora Nacional de las Fiscalias 
Especializada s en Materia Ambiental (FEMA) del Ministerio 
Publico  
Actores terciarios: Secretaria de Gestion Social y Dialogo de la 
Presidencia del Consejo de Ministros (SGSD -PCM), Defensoria 
del Pueblo.  NO HAY DIALOGO  
El 09 de agosto, el Gobierno Territorial Autono

In [93]:
texto = results['Amazonas']
texto = texto.split("|")
texto = texto[1:]
keyList = ["Dpto", "Tipo", "Ingreso como caso nuevo", "Caso", "Ubicacion","Actores primarios", "Actores secundarios", "Actores terciarios","Diálogo", "Detalle del mes"]

dptoList = []
tipoList = []
ingresoList = []
casoList = []
ubiList = []
actPrimList = []
actSecList = []
actTerList = []
dialogList = []
detalleList = []

nConflictos = int(len(texto)/9)

nConf = 0
for key in range(0, nConflictos):
    dptoList.append("Amazonas")
    tipoList.append(texto[key*9])
    ingresoList.append(texto[key*9 + 1])
    casoList.append(texto[key*9 + 2])
    ubiList.append(texto[key*9 + 3])
    actPrimList.append(texto[key*9 + 4])
    actSecList.append(texto[key*9 + 5])
    actTerList.append(texto[key*9 + 6])
    dialogList.append(texto[key*9 + 7])
    detalleList.append(texto[key*9 + 8])

In [135]:
stopWords=["\nTipo:","\nIngreso como caso nuevo:", "\nCaso:","\nUbicacion:","\nActores primarios:","\nActores secundarios:","\nActores terciarios:", "NO HAY DIALOGO", "HAY DIALOGO"]

for word in stopWords:
    conflictosActivos1 = conflictosActivos1.replace(word, "")


parts = texto[0].split("\nTipo:")
parts = parts[1:]

parts = ["\nTipo:" + part  for part in parts[:-1]]

#for word in stopWords[1:]:
#    parts = texto.split(word)
#    result_parts = [part + word for part in parts[:-1]]
#    parts = result_parts.append(parts[-1])


In [136]:
{"\nTipo:": "",
 "\nIngreso como caso nuevo:": "",
 "\nCaso:": "",
 "\nUbicacion:": "",
 "\nActores primarios:": "",
 "\nActores secundarios:": "",
 "\nActores terciarios:": "",
 "Diálogo": "NO HAY DIALOGO / HAY DIALOGO",
 "Detalle del mes" : ""}


{'\nTipo:': '',
 '\nIngreso como caso nuevo:': '',
 '\nCaso:': '',
 '\nUbicacion:': '',
 '\nActores primarios:': '',
 '\nActores secundarios:': '',
 '\nActores terciarios:': '',
 'Diálogo': 'NO HAY DIALOGO',
 'Detalle del mes': 'HAY DIALOGO'}